In [1]:
import pandas as pd 
import numpy as np

In [2]:
doc = pd.read_csv('Dataset/CassiopéeNorm/Avion.csv')

In [3]:
def format(df):
    # On créé les nouvelles colonnes en combinant les valeurs des colonnes d'origines correspondant à l'articulation
    df['chessUpper']= df['chestUpper_T_glob'].str.cat(df['chestUpper_T_glob.1'], sep=',').str.cat(df['chestUpper_T_glob.2'], sep = ',')  
    df['rShldrBend']= df['rShldrBend_T_glob'].str.cat(df['rShldrBend_T_glob.1'], sep=',').str.cat(df['rShldrBend_T_glob.2'], sep = ',') 
    df['rForearmBend']= df['rForearmBend_T_glob'].str.cat(df['rForearmBend_T_glob.1'], sep=',').str.cat(df['rForearmBend_T_glob.2'], sep = ',')
    df['lForearmBend']= df['lForearmBend_T_glob'].str.cat(df['lForearmBend_T_glob.1'], sep=',').str.cat(df['lForearmBend_T_glob.2'], sep = ',')
    df['lHand']= df['lHand_T_glob'].str.cat(df['lHand_T_glob.1'], sep=',').str.cat(df['lHand_T_glob.2'], sep = ',')
    df['pelvis']= df['pelvis_T_glob'].str.cat(df['pelvis_T_glob.1'], sep=',').str.cat(df['pelvis_T_glob.2'], sep = ',')
    
    # On les réorganise
    df = df[['Frame', 'Time(s)', 'chessUpper', 'rShldrBend', 'rForearmBend', 'lForearmBend', 'lHand','pelvis']]
    
    # On convertit les colonnes en tableau numpy afin de pouvoir séparer les valeurs des chaines de caractère dans des listes de coordonnées
    chessUpper = df['chessUpper'].values
    rShldrBend = df['rShldrBend'].values
    rForearmBend = df['rForearmBend'].values
    lForearmBend = df['lForearmBend'].values
    lHand = df['lHand'].values
    pelvis = df['pelvis'].values
    # On sépare les valeurs
    for i in range(1,df['chessUpper'].values.shape[0]):
        chessUpper[i] = chessUpper[i].split(',')
        rShldrBend[i] = rShldrBend[i].split(',')
        rForearmBend[i] = rForearmBend[i].split(',')
        lForearmBend[i] = lForearmBend[i].split(',')
        lHand[i] = lHand[i].split(',')
        pelvis[i] = pelvis[i].split(',')
    
    #On met à jour le df et on le renvoie
    df['chessUpper']= chessUpper
    df['rShldrBend']= rShldrBend
    df['rForearmBend']=  rForearmBend
    df['lForearmBend']= lForearmBend
    df['lHand']= lHand
    df['pelvis']= pelvis
    df.drop([0], axis = 0, inplace = True)
    return df



In [4]:
import pandas as pd
import numpy as np
import struct

def to_skeleton(df, name_df):
    joints_df = df.iloc[:, 2:]
    data = joints_df.to_numpy()

    num_joints = data.shape[1]
    num_frames = data.shape[0]
    framerate = 30

    with open('./Dataset/Skeletons/'+ '{}'.format(name_df) + '.skeleton', 'wb') as f:

        f.write(struct.pack('i', num_joints))
        f.write(struct.pack('i', num_frames))
        f.write(struct.pack('i', framerate))
    

        for frame in data:
            for joint in range(num_joints):

                x = float(frame[joint][0])
                y = float(frame[joint][1])
                z = float(frame[joint][2])
            

                f.write(struct.pack('fff', x, y, z))

                f.write(struct.pack('ffff', 1, 0, 0, 0))

    f.close()


In [5]:
import struct
import numpy as np
import os
import matplotlib.pyplot as plt

def plot_skeleton(skeleton_file):

    if not os.path.exists(skeleton_file):
        raise ValueError("Le fichier skeleton n'existe pas")

    with open(skeleton_file, 'rb') as f:
  
        num_joints = struct.unpack('i', f.read(4))[0]
        num_frames = struct.unpack('i', f.read(4))[0]
        framerate = struct.unpack('i', f.read(4))[0]
        

        data = np.zeros((num_frames, num_joints, 3))
        for frame in range(num_frames):
            for joint in range(num_joints):

                x, y, z = struct.unpack('fff', f.read(12))

                f.read(16) 

                data[frame, joint, :] = np.array([x, y, z])
    

    time = np.arange(num_frames) / framerate
    

    fig, axs = plt.subplots(num_joints, 1, figsize=(8, num_joints*2), sharex=True)
    

    for joint in range(num_joints):
        axs[joint].plot(time, data[:, joint, 0], label='x')
        axs[joint].plot(time, data[:, joint, 1], label='y')
        axs[joint].plot(time, data[:, joint, 2], label='z')
        axs[joint].set_title('Joint {}'.format(joint))
        axs[joint].set_ylabel('Position (m)')
        axs[joint].legend()

    axs[-1].set_xlabel('Temps (s)')
    fig.suptitle('Coordonnées des articulations')

    plt.show()


In [6]:
import os 

counter = 0

for file_name in os.listdir('Dataset/CassiopéeNorm'):

    name_df = "S001C001R001A" + "{:03d}".format(counter)
    counter += 1
    try:
        to_skeleton(format(pd.read_csv('Dataset/CassiopéeNorm/' + file_name)), name_df)

    except Exception as e :
        pass
    
    

/var/folders/ym/71jgxz4j2rz863lgsp_17w240000gn/T/ipykernel_12622/4076484993.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chessUpper']= chessUpper
/var/folders/ym/71jgxz4j2rz863lgsp_17w240000gn/T/ipykernel_12622/4076484993.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rShldrBend']= rShldrBend
/var/folders/ym/71jgxz4j2rz863lgsp_17w240000gn/T/ipykernel_12622/4076484993.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in